In [2]:
# Generates and Saves Comparison of Original to Heatmaps in .jpg format

# Import necessary modules
import ultralytics
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image

# Load your YOLO model
model = YOLO('C:\\Users\\MaxwellLee\\PycharmProjects\\UVSM\\models\\best.pt')
model.cpu()

# Prepare the input image
file = 'C:\\Users\\MaxwellLee\\PycharmProjects\\UVSM\\yolo-dataset_export_export\\test\\images\\0d9b0bf269.jpg'
img = cv2.imread(file)
img = cv2.resize(img, (320, 320))

rgb_img = img.copy()  # For visual comparison
img = np.float32(img) / 255

# Extract all layers from the model
all_layers = list(model.model.model)  # Access all layers in the model

# Create a directory to save heatmaps (optional)
import os

name = os.path.splitext(os.path.basename(file))[0]
output_dir = "C:\\Users\\MaxwellLee\\PycharmProjects\\UVSM\\runs\\heatmaps\\" + name
print(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Iterate through the first 20 layers and generate heatmaps
for i, layer in enumerate(all_layers[-20:]):  # Limit to first 20 layers
    print(f"Processing Layer {i+1}/{min(20, len(all_layers))}: {layer.__class__.__name__}")

    # Use the layer as the target layer
    target_layers = [layer]
    cam = EigenCAM(model, target_layers, task='od')

    # Generate the CAM heatmap
    grayscale_cam = cam(rgb_img)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)

    # Save or display the heatmap
    output_path = os.path.join(output_dir, f"layer_{i+1}_heatmap.jpg")
    Image.fromarray(np.hstack((rgb_img, cam_image))).save(output_path)
    print(f"Saved heatmap for Layer {i+1} to {output_path}")

print("Heatmaps generation for first 20 layers complete!")

C:\Users\MaxwellLee\PycharmProjects\UVSM\runs\heatmaps\0d9b0bf269
Processing Layer 1/20: Conv

0: 640x640 1 catalytic convertor, 1 fuel tank, 1 silencer, 1 spare tyre, 4 wheels, 145.0ms
Speed: 2.0ms preprocess, 145.0ms inference, 14.0ms postprocess per image at shape (1, 3, 640, 640)


ValueError: `k` must be an integer satisfying `0 < k < min(A.shape)`.

In [ ]:
# ################## SHOW ALL LAYERS OF SAVED HEATMAP (ASCENDING ORDER) ##################

import os
import cv2
import matplotlib.pyplot as plt

def extract_numeric_part(filename):
    # Extract numeric part from the filename (assuming the numeric part is at the end of the filename before extension)
    return int(''.join(filter(str.isdigit, filename)))  # Extract digits and convert to integer

image_path = "heatmaps/0e9dd3deea"

# List all files in the 'heatmaps' directory
image_files = os.listdir(image_path)

# Sort files numerically using the custom sorting function
image_files.sort(key=extract_numeric_part)

# Loop through each image file in the directory
for image_name in image_files:
    image_file_path = os.path.join(image_path, image_name)  # Create full path to the image file
    if os.path.isfile(image_file_path):  # Check if it's a file
        im = cv2.imread(image_file_path)  # Read the image
        print(f"{image_file_path}")

        if im is not None:
            # Convert BGR to RGB (OpenCV loads images in BGR by default)
            im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            
            # Set the figure size to enlarge the image (width, height in inches)
            plt.figure(figsize=(12.8, 6.4))  # You can adjust these values to your preference
            
            # Display image using matplotlib
            plt.imshow(im_rgb)
            plt.axis('off')  # Hide axes
            plt.show()  # Show the image in the notebook
        else:
            print(f"Failed to load image: {image_file_path}")